# 필요 모듈 정리

In [1]:
%pip install selenium
%pip install webdriver-manager

   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   --------- ------------------------------ 2.4/10.5 MB 73.9 MB/s eta 0:00:01
   --------------------- ------------------ 5.6/10.5 MB 59.0 MB/s eta 0:00:01
   ---------------------------------- ----- 9.0/10.5 MB 63.9 MB/s eta 0:00:01
   ---------------------------------------  10.5/10.5 MB 72.6 MB/s eta 0:00:01
   ---------------------------------------- 10.5/10.5 MB 50.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/467.2 kB ? eta -:--:--
   ---------------------------------------  460.8/467.2 kB ? eta -:--:--
   ---------------------------------------  460.8/467.2 kB ? eta -:--:--
   ---------------------------------------  460.8/467.2 kB ? eta -:--:--
   ---------------------------------------  460.8/467.2 kB ? eta -:--:--
   ---------------------------------------  460.8/467.2 kB ? eta -:--:--
   ---------------------------------------- 467.2/467.2 kB 1.7 MB/s eta 0:00:00
   -------------------

DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import requests
from bs4 import BeautifulSoup
import json
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# jobplanet 페이지 구조

- https://www.jobplanet.co.kr/companies/30139/reviews/삼성전자
    - span[class="rate_point] : 전체 리뷰 평점
    - span[class="txt_point"] : 순서대로 [복지 및 급여, 업무와 삶의 균형, 사내문화, 승진 기회 및 가능성, 경영진, 기업 추천율, ceo 지지율, 성장 가능성]

- https://www.jobplanet.co.kr/companies/30139/benefits/삼성전자
    - span[class="welfare-star__point] : 전체 복지 평점
    - button[class="welfare-overview__num"] : 복지 개수


# jobplanet 기업 고유번호 가져오기
- selenium 이용
- 구글 -> 잡플래닛 + 기업명 검색
- 구글 로봇 정책을 피하기 위해서 10개씩 나누어 jobplanet_corp_code.json 저장

In [3]:
# 기업명을 반복문 돌리기 위한 kospi200 기업 리스트 만들기
with open('top200_name_list.csv', 'r', encoding= 'utf-8') as file :
    kospi_200_corp_list = [corp_name.strip() for corp_name in file.readlines()]

In [77]:
# corp_code 정리를 위한 딕셔너리 생성
job_planet_corp_code = {}
crawling_failed_list = []

In [78]:
for corp in kospi_200_corp_list[190:200] : 
    # 1. 구글 사이트 접속하기
    driver.get('https://www.google.com')

    # 2. 검색창에 커서 놓기
    search_part = driver.find_element(By.CSS_SELECTOR, 'textarea[class="gLFyf"]')

    # 3. 잡플래닛 + 기업명 검색어 입력하기
    search_part.send_keys(f'잡플래닛 {corp}')

    # 4. 구글 검색 아이콘 클릭하기
    driver.find_elements(By.CSS_SELECTOR, 'input[role="button"]')[1].click()

    time.sleep(1)

    # 5. selenium을 통해 동적 html 소스 가져오기
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    corp_code_htmls = soup.select('a[jsname="UWckNb"]')

    html_tags = ''
    for corp_code in corp_code_htmls :
        html_tags += str(corp_code)
    
    if re.findall('https://www.jobplanet.co.kr/companies/[0-9]+', html_tags) != [] :
        corp_code = int(re.findall('https://www.jobplanet.co.kr/companies/[0-9]+', html_tags)[0].split('/')[-1])
        job_planet_corp_code[corp] = corp_code
    else :
        crawling_failed_list.append(corp)
        print(f'{corp} 회사는 크롤링에 실패했습니다')
        
    
# db 금융지주에서 오류남!

In [79]:
with open('jobplanet_corp_code.json', 'a', encoding='utf-8-sig') as f : 
	json.dump(job_planet_corp_code, f, ensure_ascii=False, indent= '\t')

# 잡플래닛 회사 고유번호를 이용한 잡플래닛 크롤링
- 잡플래닛 자체가 cloudfare로 로봇 크롤링을 막아놨기 때문에 selenium으로 진행
- jobplanet_corp_code.json 불러오기

In [26]:
# 잡플래닛 내부 회사 번호 딕셔너리 불러오기
with open('./jobplanet_corp_code_크롤링 해야할 회사.json', 'r', encoding='utf-8-sig') as file :
    jp_corp_code = json.load(file)
len(jp_corp_code)

165

- 잡플래닛 로그인 하기
    - 셀레니움은 기본적으로 세션 유지가 되기 때문에 로그인 상태를 유지하면서 페이지를 바꿀 수 있다. 

In [27]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# 1. 잡플래닛 로그인 url로 접속하기
driver.get('https://www.jobplanet.co.kr/users/sign_in?_nav=gb')

In [3]:
with open('./jobplanet_config', 'r', encoding = 'utf-8') as file :
    id_pw = [config.strip() for config in file.readlines()]

id = id_pw[0]
password = id_pw[1]

['12203193@inha.edu', '24172019sh!']

In [28]:
# 2. 아이디 입력창에 커서 놓기
input_id = driver.find_element(By.CSS_SELECTOR, 'input[id="user_email"]')

# 3. 아이디 입력창에 아이디 입력
input_id.send_keys(id)

# 4. 비밀번호 입력창에 커서 놓기
input_password = driver.find_element(By.CSS_SELECTOR, 'input[id="user_password"]')

# 5. 비밀번호 입력창에 비밀번호 입력
input_password.send_keys(password)

# 6. 로그인 버튼 클릭하기
driver.find_element(By.CSS_SELECTOR, 'button[class="btn_sign_up"]').click()

# 로그인 상태를 유지함.


- 각 기업 별 <리뷰>와 <복지> 탭에 있는 정보들 크롤링하기
    - 리뷰 탭 크롤링 대상 
        - 1p : [전체 리뷰 평점, 복지 및 급여, 업무와 삶의 균형, 사내문화, 승진 기회 및 가능성, 경영진, 기업추천율, ceo 지지율, 성장 가능성] 
        - 모든 페이지 : 기업 리뷰들
    - 복지 탭 크롤링 대상
        - 1p : [전체 복지 평점, 복지 개수]
        - 모든 페이지 : 복지 리뷰들

## 최종 데이터 셋

- [판다스]   
    - 인덱스 -> jp_corp_code 기업순서   
    - 칼럼 ->   
전체 리뷰 평점, 복지 및 급여, 업무와 삶의 균형, 사내문화, 승진 기회 및 가능성, 경영진, 기업추천율,   
ceo 지지율, 성장 가능성 ,기업 리뷰들, 전체 복지 평점, 복지 개수 복지 리뷰들    
--> 기업 순서별로 각각의 리스트에 넣어주기

In [29]:
아이디, 회사순서, 전체리뷰평점,복지및급여,업무와삶의균형 ,사내문화 ,승진기회및가능성 ,경영진 ,기업추천율 ,ceo지지율 ,성장가능성, 총복지평점, 총복지개수, 장점리뷰, 단점리뷰 = [[] for _ in range(15)]

In [30]:
for i, corp_name in enumerate(jp_corp_code) :
    # if corp_name != 'CJ대한통운' : --> 기업 범위 지정할 때 사용할 로직
        # 1. 리뷰 탭에 있는 수치형 데이터 크롤링하기
        try:
            driver.get(f'https://www.jobplanet.co.kr/companies/{jp_corp_code[corp_name]}/reviews?page=1')
            
            리뷰평점 = float(driver.find_element(By.CSS_SELECTOR, 'span[class="rate_point"]').text)
            
            review_datas = driver.find_elements(By.CSS_SELECTOR, 'span[class="txt_point"]')
            review_composition = [float(review_data.text.replace('%', '').replace('--', '0')) if '%' in review_data.text else float(review_data.text) for review_data in review_datas]
            
            아이디.append(i+1)
            회사순서.append(corp_name)
            전체리뷰평점.append(리뷰평점)
            복지및급여.append(review_composition[0])
            업무와삶의균형.append(review_composition[1])
            사내문화.append(review_composition[2])
            승진기회및가능성.append(review_composition[3])
            경영진.append(review_composition[4])
            기업추천율.append(review_composition[5])
            ceo지지율.append(review_composition[6])
            성장가능성.append(review_composition[7])
        except :
            아이디.append(i+1)
            회사순서.append('NULL')
            전체리뷰평점.append('NULL')
            복지및급여.append('NULL')
            업무와삶의균형.append('NULL')
            사내문화.append('NULL')
            승진기회및가능성.append('NULL')
            경영진.append('NULL')
            기업추천율.append('NULL')
            ceo지지율.append('NULL')
            성장가능성.append('NULL')
        


        # 2. 복지 탭에 있는 수치형 데이터 크롤링하기
        driver.get(f'https://www.jobplanet.co.kr/companies/{jp_corp_code[corp_name]}/benefits')
        try : # 복지 종합 데이터가 존재할 경우
            복지평점 = driver.find_element(By.CSS_SELECTOR, 'span[class="welfare-star__point"]').text
            복지개수 = driver.find_element(By.CSS_SELECTOR, 'button[class="welfare-overview__num"]').text
            총복지평점.append(복지평점)
            총복지개수.append(복지개수)
        except :
            총복지평점.append('NUll')
            총복지개수.append('NUll')
        
        # 3. 리뷰 탭에 있는 장단점 텍스트 데이터 크롤링하기 
        concat_pos = '' 
        concat_neg = ''
        
        for page_num in range(1,150) :
            try : # dd[class="df1"]가 없을 수 있다. 즉 마지막 페이지까지 지났을 때 대비하기
                driver.get(f'https://www.jobplanet.co.kr/companies/{jp_corp_code[corp_name]}/reviews?page={page_num}&year%5B%5D=2024&year%5B%5D=2023&year%5B%5D=2022&year%5B%5D=2021')
                employee_talks = driver.find_elements(By.CSS_SELECTOR, 'dd[class="df1"]')

                
                pos = [posi.text.replace('\n', ' ').strip() for i, posi in enumerate(employee_talks) if i % 3 == 0 ]
                neg = [nega.text.replace('\n', ' ').strip() for i, nega in enumerate(employee_talks) if i % 3 == 1 ]


                concat_pos += '||'.join(pos) # '||'.join(pos) : 한 페이지에 대해서 장점을 모은 것
                concat_neg += '||'.join(neg) # concat_pos/neg : 모든 페이지의 장점과 단점을 모은 것 <- 장점/대상 리뷰 append 대상
            except :
                print(f'{corp_name}의 {page_num}에서 크롤링을 멈췄습니다. 페이지 범위가 아닌 다른 문제일 수 있으니 몇 개는 직접 확인해보세요')
        장점리뷰.append(concat_pos)
        단점리뷰.append(concat_neg)
            
    # else : break
        
        


print('크롤링 완료')

    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span[class="rate_point"]"}
  (Session info: chrome=123.0.6312.105); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x009E4CC3+225091]
	(No symbol) [0x00914E11]
	(No symbol) [0x007B9A7A]
	(No symbol) [0x007F175B]
	(No symbol) [0x007F188B]
	(No symbol) [0x00827882]
	(No symbol) [0x0080F5A4]
	(No symbol) [0x00825CB0]
	(No symbol) [0x0080F2F6]
	(No symbol) [0x007E79B9]
	(No symbol) [0x007E879D]
	sqlite3_dbdata_init [0x00E59A63+4064547]
	sqlite3_dbdata_init [0x00E6106A+4094762]
	sqlite3_dbdata_init [0x00E5B968+4072488]
	sqlite3_dbdata_init [0x00B5C9C9+930953]
	(No symbol) [0x009207E4]
	(No symbol) [0x0091AD08]
	(No symbol) [0x0091AE31]
	(No symbol) [0x0090CAA0]
	BaseThreadInitThunk [0x76E3FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77307C5E+286]
	Rt

In [31]:
jp_df = pd.DataFrame({'회사명' : 회사순서, '복지평점' : 총복지평점, '복지개수' : 총복지개수, '리뷰평점' : 전체리뷰평점, '복지 및 급여' : 복지및급여,
                      '워라밸' : 업무와삶의균형, '사내문화' : 사내문화, '승진기회및가능성' : 승진기회및가능성, '경영진 점수' : 경영진, '기업추천율' : 기업추천율,
                      'ceo지지율' : ceo지지율, '성장가능성' : 성장가능성, '직원들이 평가한 장점' : 장점리뷰, '직원들이 평가한 단점' : 단점리뷰 })

In [32]:
jp_df

,회사명,복지평점,복지개수,리뷰평점,복지 및 급여,워라밸,사내문화,승진기회및가능성,경영진 점수,기업추천율,ceo지지율,성장가능성,직원들이 평가한 장점,직원들이 평가한 단점
0,LG전자,3.8,76,3.1,3.0,3.1,3.2,3.0,2.5,49.0,50.0,22.0,정규직 아니고 파견직으로 업무하고 있습니다. 사람들 친절하고 일도 무리하지 않습니다...,딱히 없음. 업무 강도가 부서에 따라 조금씩 다른 듯.||일이 많고 신입사원들에게 ...
1,LG화학,4.0,63,3.5,3.3,3.5,3.4,3.4,2.9,64.0,60.0,44.0,착한 사람들 제도적으로는 보장되는 워라벨 찻한 사람들이 있음||대기업인 만큼 어느정...,직원하대 보고를 위한 보고 정말 답답한 문화 멍청이들 천국||공장이다보니 옛날 문화...
2,LIG넥스원,3.7,64,3.7,3.7,3.5,3.6,3.4,2.7,65.0,55.0,31.0,"복지가 좋고, 회사 위치 및 분위가 좋음. 팀바이팀이겠지만 연구 위주의 회사||꾸준...","출장이 많고, 직급이 높아짐에 따라 연봉 상승률이 낮음.||잦은 출장으로 중간 관리..."
3,LS,NUll,NUll,5.0,5.0,5.0,4.0,5.0,4.0,100.0,0.0,100.0,,
4,LS ELECTRIC,3.7,45,3.4,2.8,4.2,3.4,2.5,2.7,54.0,43.0,19.0,1. 워라벨 2. 괜찮은 신입 초봉 3. 특별히 매우 나쁜건 없이 무난||수평적인 ...,1. 의미없는 회의와 보고서가 너무 많음 2. 진급할수록 연봉이 안올라 과장부터는 ...
5,LX인터내셔널,4.1,38,3.1,3.4,3.1,2.9,3.2,2.5,44.0,45.0,17.0,"연봉이 나쁘지 않음, 최근 시황 나쁘지 않음, 과일컵 줌||해외영업이나 무역 직무를...","상상초월 꼰대들, 심심하면 터지는 추문, 미쳐버린 손익추정||하지만... 석탄 말고..."
6,NAVER,4.4,67,4.0,4.1,3.5,3.8,3.5,3.2,85.0,70.0,57.0,전반적으로 잘 마련된 시스템. 특히 재택 근무 선택 방식은 정말 필요하고 효율성을 ...,"리더, 경영진 교육을 하겠지만 소문이나 평이 안 좋은 곳은 피하고 싶음. 시스템 안..."
7,NH투자증권,4.0,36,3.8,4.2,3.7,3.4,3.3,3.3,77.0,76.0,45.0,"복지가 매우 좋고 정규직은 보장 매우 잘되어있음 꿀빨 수 있는듯||부바이부겠지만, ...",사장이 항상 바뀌면서 사업 연속성이 매우 떨어지고 그에 따른 내부 정치질 매우 심함...
8,OCI홀딩스,3.1,63,3.1,2.8,3.5,2.9,2.9,2.3,36.0,35.0,14.0,"사업 아이템이 어느정도 경기를 타긴 하지만, 최근 국제정세 흐름에 따라 어느정도 수...","신규사업의 방향성을 잘 정해야 하는데, 아직까지는 오너 중심으로만 의사결정이 이루어..."
9,PI첨단소재,3.6,53,3.2,3.9,2.9,2.4,3.4,2.3,49.0,33.0,30.0,"없음, 그나마 비급여 처리되는 의료비 예전에는 그나마 성과급을 챙겨줬지만 요즘에 그...","사람, 사람, 사람 당신이 인간이라면 여기서 사람 때문에 엄청난 스트레스를 겪을 것..."


In [33]:
jp_df.to_csv('jobplanet_output35-51.csv')